# Advanced

In [14]:
import numpy as np
from roseau.load_flow import *

Network creation

In [15]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0

# Create a MV voltage source
un = 20e3 / np.sqrt(3)
voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, voltages=voltages)

# Add a transformer
bus0 = Bus(id="bus0", n=4)
tc = TransformerCharacteristics("160_kVA", "Dyn11", sn=160.0 * 1e3, uhv=20e3, ulv=400.0, i0=2.3 / 100, p0=460.0, psc=2350.0, vsc=4.0 / 100)
transformer = DeltaWyeTransformer(id="transfo", bus1=vs, bus2=bus0, transformer_characteristics=tc, tap=1.025)
ground.connect(bus0)

# Add a LV network
lc = LineCharacteristics.from_name_lv("S_AL_150")
bus1 = Bus(id="bus1", n=4)
bus2 = Bus(id="bus2", n=4)
load_bus1 = Bus(id="load_bus1", n=4)
load_bus2 = Bus(id="load_bus2", n=4)
load_bus3 = Bus(id="load_bus3", n=4)
line1 = ShuntLine(id="line1", n=4, bus1=bus0, bus2=bus1, ground=ground, line_characteristics=lc, length=0.5)
line2 = ShuntLine(id="line2", n=4, bus1=bus1, bus2=bus2, ground=ground, line_characteristics=lc, length=0.4)
line3 = ShuntLine(id="line3", n=4, bus1=bus1, bus2=load_bus1, ground=ground, line_characteristics=lc, length=0.3)
line4 = ShuntLine(id="line4", n=4, bus1=bus2, bus2=load_bus2, ground=ground, line_characteristics=lc, length=0.3)
line5 = ShuntLine(id="line5", n=4, bus1=load_bus2, bus2=load_bus3, ground=ground, line_characteristics=lc, length=0.4)
si = -3e10
load1 = PowerLoad(id="load1", n=4, bus=load_bus1, s=[si, si, si])
load2 = PowerLoad(id="load2", n=4, bus=load_bus2, s=[si, si, si])
load3 = PowerLoad(id="load3", n=4, bus=load_bus3, s=[si, 0, 0])

# Create the network
en = ElectricalNetwork.from_element(vs)

In [16]:
en.solve_load_flow(auth=("username", "dev password"))
np.abs(load_bus3.potentials[0:3] - load_bus3.potentials[3])

Magnitude,[249.16095029692727 237.94587246515812 239.2552125033545]
Units,volt


Flexible loads usage

In [17]:
# Let's make the load 3 flexible with a p(u) control to reduce the voltages constraints
en.remove_element("load3")
parameter = FlexibleParameter.p_max_u_production(u_up=240, u_max=250, s_max=4000)
flexible_load = FlexibleLoad(
    id="load3", n=4, bus=load_bus3, s=[si, 0, 0], parameters=[parameter, FlexibleParameter.constant(), FlexibleParameter.constant()]
)
en.add_element(flexible_load)

In [18]:
en.solve_load_flow(auth=("username", "dev password"))
np.abs(load_bus3.potentials[0:3] - load_bus3.potentials[3])

Magnitude,[245.89349655674107 239.36821845653907 239.70464920201982]
Units,volt


In [19]:
flexible_load.powers

Magnitude,[-1628.91699011+0.j 0. +0.j 0. +0.j]
Units,volt_ampere


In [20]:
# Let's try with pq(u) control, by injecting reactive power before reducing active power
en.remove_element("load3")
parameter = FlexibleParameter.pq_u_production(up_up=240, up_max=250, uq_min=200, uq_down=210, uq_up=235, uq_max=240, s_max=4000)
flexible_load = FlexibleLoad(
    id="load3", n=4, bus=load_bus3, s=[si, 0, 0], parameters=[parameter, FlexibleParameter.constant(), FlexibleParameter.constant()]
)
en.add_element(flexible_load)

In [21]:
en.solve_load_flow(auth=("username", "dev password"))
np.abs(load_bus3.potentials[0:3] - load_bus3.potentials[3])

Magnitude,[243.5587627848849 236.24816264943718 243.8014280349357]
Units,volt


In [22]:
flexible_load.powers

Magnitude,[-2076.69942528+3418.67217664j 0. +0.j 0. +0.j ]
Units,volt_ampere
